# IT Academy - Data Science amb Python
## Tasca 7: Estructura de control

###  [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)

###  Exercici 1
- Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [ ]:
import numpy as np
import pandas as pd
import requests
import pickle
import json
import time
import re
import collections, numpy

from datetime import datetime
from datetime import timedelta
from joblib import Parallel, delayed
from urllib.request import urlopen
import collections, numpy

pd.set_option('display.max_columns', None)

In [ ]:
def counter(routine):
    start = time.monotonic()
    routine
    end = time.monotonic()
    print(timedelta(seconds = end - start))

In [ ]:
path = '../Data/Web_access_log-akumenius.com.txt'

Logs_raw = pd.read_csv(path, sep='\s | \- | \"', names =['DNS','IP','Location','Date','Time','Request','Status','Size','Referer','UserAgent'], engine='python')

Logs_copy = Logs_raw.copy()

Logs_copy.head()

###  Exercici 2
- Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [ ]:
Logs_copy.tail()

In [ ]:
Logs_copy.shape

In [ ]:
Logs_copy.isnull().sum()

In [ ]:
Logs_copy.head()

In [ ]:
Logs_copy.tail()

- Reorder columns of data

In [ ]:
Logs_copy.UserAgent = Logs_copy.Time
Logs_copy.Request = Logs_copy.Location
Logs_copy.Referer = Logs_copy.Date
Logs_copy.Date = Logs_copy.IP
Logs_copy.Time = Logs_copy.IP = Logs_copy.Location = np.nan

Logs_copy.head()

- DNS & IP data split

In [ ]:
Logs_copy[['DNS','IP']] = Logs_copy.DNS.str.split('\s', expand = True).get([0, 1])

In [ ]:
Logs_copy.head()

In [ ]:
Logs_copy.DNS.unique()

In [ ]:
len(Logs_copy.IP.unique())

- Time data extraction

In [ ]:
Logs_copy.Time = Logs_copy.Date.str.extract(':(\d{2}:\d{2}:\d{2}.*)]')

#Logs_copy.Time = pd.to_datetime(Logs_copy.Time, format = '%H:%M:%S %z')
Logs_copy.head()

- Date data extraction

In [ ]:
Logs_copy.Date = Logs_copy.Date.str.extract('(\d+/\w+/\d+)')

#Logs_copy.Date = pd.to_datetime(Logs_copy.Date, format = '%d/%b/%Y')

In [ ]:
Logs_copy.head()

- Request & Status data split

In [ ]:
Logs_copy[['Request','Status']] = Logs_copy.Request.str.split('\"', expand = True).get([0, 1])

In [ ]:
Logs_copy.tail()

In [ ]:
Logs_copy.head()

- Size data extraction

In [ ]:
Logs_copy.Size = Logs_copy.Status.str.extract('(\d+$)')
Logs_copy.head()

- Clean Size data

In [ ]:
sizes = []

for index, row in Logs_copy.iterrows(): 
    
    #if re.search('200', row.Size):
    if row.Size == '200':
        sizes.append(np.nan)
    else:
        sizes.append(row.Size)

Logs_copy.Size = sizes

Logs_copy.head()

- Status data extraction

In [ ]:
Logs_copy.Status = Logs_copy.Status.str.extract('(\d{3})')
Logs_copy.tail()

- Clean Referer data

In [ ]:
%%time
referers = []

for index, row in Logs_copy.iterrows(): 
    
    if re.search('-"', row.Referer):
        referers.append(np.nan)
    else:
        referers.append(row.Referer.rstrip(row.Referer[-1]))

Logs_copy.Referer = referers

In [ ]:
Logs_copy

###  Exercici 3
- Geolocalitza les IP's. Aqui tens una pagina de interes:
  - [freegeoip](https://freegeoip.app/)

- We export Ips file for security

In [ ]:
Ips_export= Logs_copy.IP.copy()

Ips_export.replace('', 'null', inplace = True)

Ips_export.to_csv('../Data/Ips_export.csv', index = False)

- Ips file recovery

In [ ]:
path = '../Data/Ips_export.csv'

Ips_raw = pd.read_csv(path, sep= 'delimiter', engine='python')

Ips_raw.head()

In [ ]:
Ips_raw.tail()

In [ ]:
Ips_copy = Ips_raw.copy()
ips_unique = Ips_copy.IP.unique()

In [ ]:
len(Ips_copy.IP.unique())

In [ ]:
ips_unique

- Function for extract Locations freegeoip

In [ ]:

localhost = "127.0.0.1"
loalhost_info ={'ip': '127.0.0.1',
 'country_code': 'ES',
 'country_name': 'Spain',
 'region_code': '',
 'region_name': '',
 'city': '',
 'zip_code': '',
 'time_zone': '',
 'latitude': '',
 'longitude': '',
 'metro_code': ''}

def extract_info(ip):

    try:
        if ip == localhost:
            return localhost_info
        else:
            response = urlopen("https://freegeoip.app/json/" + ip)
            data = json.load(response)
            return data  

    except:
        return np.nan

In [ ]:
extract_info(ips_unique[1])

- Ips Information Extraction

In [ ]:
%%time

Info_list = Parallel(n_jobs = 8, backend = "multiprocessing")(map(delayed(extract_info), ips_unique[0]))

- joblib.Parallel uses the backend module to start worker processes, executing tasks simultaneously on separate CPUs, obtaining the IPS 2921 extraction time 8 times faster, when using 8 cores

In [ ]:
Info_list[1]

In [ ]:
len(Info_list)

In [ ]:
# File to pickle
def to_pickle(info):
    with open('geolocation.pickle', 'wb') as file:
        pickle.dump(info, file)

- We export pickle info for security

In [ ]:
to_pickle(Info_list)

- Pickle file recovery

In [ ]:
ips_info = pd.read_pickle('geolocation.pickle')

In [ ]:
Info = pd.DataFrame(ips_unique, columns=['IP'])
Info.head()

In [ ]:
len(Info)

In [ ]:
Info['Info'] = ips_info
Info

In [ ]:
Logs_copy = Logs_copy.merge(Info, on='IP', how='inner')

In [ ]:
Logs_copy

###  Exercici 4
- Mostreu-me la teva creativitat, Sorprèn-me fes un pas més enllà amb el analysis anterior.

In [ ]:
- Extract UserAgent Data

In [ ]:
from device_detector import SoftwareDetector

Devices = Logs_copy.UserAgent
device = []

for x in Devices:
    
    device.append(SoftwareDetector(x).parse())

client_name = []
client_type = []
client_version = []
os_name = []
os_version = []
device_type = []

for x in device:
    
    client_name.append(x.client_name())
    client_type.append(x.client_type())
    client_version.append(x.client_version())
    os_name.append(x.os_name())
    os_version.append(x.os_version())
    device_type.append(x.device_type())

Logs_copy['Client_Name'] = client_name
Logs_copy['Client_Type'] = client_type
Logs_copy['Client_Version'] = client_version
Logs_copy['Os_Name'] = os_name
Logs_copy['Os_Version'] = os_version
Logs_copy['Device_Type'] = device_type

Logs_copy

In [ ]:
Logs_copy.UserAgent[0]

In [ ]:
del(Logs_copy['UserAgent'])

Logs_copy.tail()

In [ ]:
- Clean Client Data

In [ ]:
client = []

for index, row in Logs_copy.iterrows(): 
    
    if re.search('Apache', row.Client_Name):
        client.append(row.Client_Name[0:6])
    else:
        client.append(row.Client_Name)

Logs_copy.Client_Name = client

Logs_copy.head()

In [ ]:
Logs_copy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:

Logs_copy.Location.value_counts(normalize=False).where(Logs_copy.Location.value_counts() > 2000).plot(kind = 'pie', figsize = (10,6))

In [ ]:
graph = Logs_copy[['DNS', 'Location']].groupby(['DNS']).count().sort_values(by = 'Location',ascending = False)
graph = graph.rename(columns = {'Location' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'g', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
graph = Logs_copy[['Status', 'Time']].groupby(['Status']).count().sort_values(by = 'Time',ascending = False)
graph = graph.rename(columns = {'Time' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'r', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
graph = Logs_copy[['Client_Type', 'Time']].groupby(['Client_Type']).count().sort_values(by = 'Time',ascending = False)
graph = graph.rename(columns = {'Time' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'y', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
sns.displot(data = Logs_copy, x = "Os_Name", hue = "Date", multiple = "stack")

In [ ]:
sns.displot(Logs_copy.Os_Name)

In [ ]:
sns.displot(data = Logs_copy, x = "Device_Type", hue = "Date", multiple = "stack")

In [ ]:
Logs_export = Logs_copy.copy()

Logs_export.replace('', 'null', inplace = True)

Logs_export.to_csv('../Data/Logs_export.csv', index = False)